In [ ]:
import mido
from mido import Message, MidiFile, MidiTrack
import pretty_midi
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time

## helper functions

In [ ]:
def calculate_delta_time(start_time, current_time, ticks_per_beat):
    # Calculate delta time in seconds
    delta_seconds = current_time - start_time
    # Convert delta time to ticks
    return int(delta_seconds * ticks_per_beat)

## setup

In [ ]:
print(f"found input ports: {mido.get_input_names()}")
print(f"found output ports: {mido.get_output_names()}")

infile = 'files/transpose-060-01_0000-0005.mid'
outfile = 'files/live_piano_recording.mid'

inport = mido.get_input_names()[0]
outport = mido.get_output_names()[0]

In [ ]:
# Create a new MIDI file and track
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
track.append()

ticks_per_beat = mid.ticks_per_beat  # Default ticks per beat for MIDI file
start_time = -1
end_time = -1
last_note_time = start_time

# Open the input port for the MIDI keyboard
with mido.open_input(inport) as inport:
    print(f"recording at {ticks_per_beat}tpb... Press Ctrl+C to stop.")

    try:
        for msg in inport:
            current_time = time.time()
            delta_time = calculate_delta_time(last_note_time, current_time, ticks_per_beat)
            msg.time = delta_time
            print(msg)
            last_note_time = current_time 
            if msg.type in ['note_on', 'note_off']:
                print("adding note to track: {msg}")
                track.append(msg)
    except KeyboardInterrupt:
        # Stop recording on Ctrl+C
        end_time = time.time()
        print(f"stopping recording at {end_time}...")

# Save the recorded MIDI messages to a file
mid.save(outfile)
print("Recording saved to 'live_piano_recording.mid'.")


In [ ]:
def print_midi(filename):
    midi_file = MidiFile(filename)
    for i, track in enumerate(midi_file.tracks):
            print(f'=== Track {i}')
            for message in track:
                print(f'\t{message!r}')
print_midi(outfile)

In [ ]:
def draw_midi(midi_file: str, labels: bool = False):
    plt.style.use("dark_background")

    midi = pretty_midi.PrettyMIDI(midi_file)

    _, ax = plt.subplots(figsize=(12, 4))

    for note in midi.instruments[0].notes:
        rect = patches.Rectangle(
            (note.start, note.pitch), note.end - note.start, 1, color="green"
        )
        ax.add_patch(rect)

    if labels:
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("MIDI Note")
    ax.set_yticks([])
    ax.set_title(f"{Path(midi_file).stem}")

    plt.box(False)
    plt.ylim(20, 108)  # MIDI note range for a piano
    plt.xlim(0, np.ceil(midi.instruments[0].notes[-1].end))
    plt.show()

In [ ]:
draw_midi(outfile)

In [ ]:
def play_midi_file(midi_file_path, output_port_name):
    """
    Play a MIDI file through the specified MIDI output port.

    Parameters:
    - midi_file_path: The path to the MIDI file to be played.
    - output_port_name: The name of the MIDI output port.
    """
    # Load the MIDI file
    mid = MidiFile(midi_file_path)
    
    # Open the MIDI output port
    with mido.open_output(output_port_name) as outport:
        print(f"Playing {midi_file_path} on {output_port_name}...")
        for msg in mid.play():
            if not msg.is_meta:
                outport.send(msg)

    print("Playback finished.")

In [ ]:
play_midi_file('../recordings/live_piano_recording.mid', 'Disklavier')